In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
types_names = {90:'Ia', 67: '91bg', 52:'Iax', 42:'II', 62:'Ibc', 
               95: 'SLSN', 15:'TDE', 64:'KN', 88:'AGN', 92:'RRL', 65:'M-dwarf',
               16:'EB',53:'Mira', 6:'MicroL', 991:'MicroLB', 992:'ILOT', 
               993:'CART', 994:'PISN',995:'MLString'}

SNANA_types = {90:11, 62:{1:3, 2:13}, 42:{1:2, 2:12, 3:14},
               67:41, 52:43, 64:51, 95:60, 994:61, 992:62,
               993:63, 15:64, 88:70, 92:80, 65:81, 16:83,
               53:84, 991:90, 6:{1:91, 2:93}}

SNANA_names = {11: 'Ia', 3:'Ibc', 13: 'Ibc', 2:'II', 12:'II', 14:'II',
               41: '91bg', 43:'Iax', 51:'KN', 60:'SLSN', 61:'PISN', 62:'ILOT',
               63:'CART', 64:'TDE', 70:'AGN', 80:'RRL', 81:'M-dwarf', 83:'EB',
               84:'Mira', 90:'MicroLB', 91:'MicroL', 93:'MicroL'}

In [ ]:
k = 5
cases = os.listdir('/media/emille/git/COIN/RESSPECT_work/PLAsTiCC/' + \
                   'metrics_paper/' + \
                   'resspect_metric/SALT2_fit/WFD' + str(k) + '/')

In [ ]:
#cases.remove('fiducial.csv')
#cases.remove('random.csv')
#cases.remove('non-survivors')
#cases.remove('fiducial6000fail5999.csv')
#cases.remove('perfect6000.csv')
cases.remove('.ipynb_checkpoints')
#cases.remove('3000_0.csv')
#cases.remove('75SNIa25SNIax.csv')
if k == '':
    cases.remove('perfect1500.csv')
    cases.remove('perfect6000.csv')
    cases.remove('perfect3000_IX.csv')
    cases.remove('perfect3000_I.csv') 
    cases.remove('perfect3000_II.csv')
    cases.remove('perfect3000_III.csv')
    cases.remove('perfect3000_IV.csv')
    cases.remove('perfect3000_V.csv')
    cases.remove('perfect3000_VI.csv')
    cases.remove('perfect3000_VII.csv')
    cases.remove('perfect3000_VIII.csv')
    cases.remove('perfect3000_0.csv')

# check percentages

In [ ]:
for name in cases:
    
    print(name)

    fname = '/media/emille/git/COIN/RESSPECT_work/PLAsTiCC/metrics_paper/' + \
            'resspect_metric/SALT2_fit/WFD' + str(k) + '/' \
              + name 

    data = pd.read_csv(fname)

    types, freq = np.unique(data['SIM_TYPE_INDEX'].values, return_counts=True)
        
    print('\n')
    print('case: ' + name)
    for i in range(len(types)):
        print('perc ' + SNANA_names[types[i]] + ' : ', round(freq[i]/data.shape[0], 2))
    print('Total number: ', data.shape[0])
    print('\n')

# read Wassernstein distances

In [ ]:
fname = '/media/emille/git/COIN/RESSPECT_work/PLAsTiCC/metrics_paper/' + \
        'resspect_metric/Wasserstein/wassersteinDistances_wfd' + str(k) +'.dat'

wdist = pd.read_csv(fname)
name_flagA = np.array(['perfect3000' in name for name in wdist['FileA'].values])
name_flagB = np.array(['perfect3000' in name for name in wdist['FileB'].values])
name_flag = np.logical_or(name_flagA, name_flagB)

wdist_wfd = wdist[name_flag]

## calculate classification metrics

In [ ]:
import proclam
from proclam.metrics.util import *
from proclam.metrics.util import RateMatrix

In [ ]:
class det_mets(RateMatrix):
    "binary classification metrics"
    def __init__(self, **rates):
        """
        Call like `thing = det_mets(**rates._asdict())`
        """
#         self.rates = rates#.asdict()
        self._get_tots()
        self._from_rates()
        self._sn_mets()
        self._translate()
    def _get_tots(self):
        self.CP = self.TP + self.FN
        self.CN = self.TN + self.FP
        self.T = self.TP + self.TN
        self.F = self.FP + self.FN
        self.P = self.TP + self.FP
        self.N = self.TN + self.FN
    def _from_rates(self):
        self.PPV = self.TP / (self.TP + self.FP)
        self.NPV = self.TN / (self.TN + self.FN)
        self.PT = (np.sqrt(self.TPR * (1. - self.TNR)) + self.TNR - 1.) / (self.TPR + self.TNR - 1.)
        self.TS = self.TP / (self.TP + self.FN + self.FP)
        self._derived()
    def _derived(self):
        self.ACC = (self.TP + self.TN) / (self.CP + self.CN)
        self.BA = (self.TPR + self.TNR) / 2,
        self.F1S = 2. * self.PPV * self.TPR / (self.PPV + self.TPR)
        self.MCC = (self.TP * self.TN - self.FP * self.FN) / (np.sqrt(self.P * self.CP * self.CN * self.N))
        self.FM = np.sqrt(self.PPV * self.TPR)
        self.BM = self.TPR + self.TNR - 1.
        self.MK = self.PPV + self.NPV - 1.
    def _translate(self):
        self.positive = self.CP
        self.negative = self.CN
        self.sensitivity = self.TPR
        self.recall = self.TPR
        self.specificity = self.TNR
        self.selectivity = self.TNR
        self.precision = self.PPV
        self.FDR = 1. - self.PPV
        self.FOR = 1. - self.NPV
        self.CSI = self.TS
        self.accuracy = self.ACC
        self.f1_score = self.F1S
        self.informedness = self.BM
        self.deltaP = self.MK
    def _sn_mets(self):
        self.get_efficiency()
        self.get_purity()
    def get_efficiency(self):
        self.efficiency = self.TP / self.CP
        return self.efficiency
    def get_purity(self):
        self.purity = self.TP / self.P
        return self.purity
    def get_fom(self, penalty):
        self.pseudo_purity = self.TP / (self.TP + penalty * self.FP)
        return self.pseudo_purity * self.efficiency

In [ ]:
def nums_to_rate(tot, P, T, FP):
    N = tot - P
    TN = N - FP
    F = tot - T
    FN = F - TN
    TP = P - FN
    assert(FP + FN + TP + TN == tot)
    
    TPR = TP / P
    FPR = FP / N
    FNR = FN / P
    TNR = TN / N
    
#     cm = np.array([[totIa - cont, totall - ], [, cont]])
    rate = proclam.util.RateMatrix(TPR=TPR, FPR=FPR, FNR=FNR, TNR=TNR, TP=TP, FN=FN, TN=TN, FP=FP)
    return rate

In [ ]:
def classification_metrics(cont, totIa=3000, before_salt2=False, field='DDF'):
    """Classification metrics for a sample of 3k SNIa.
    
    Parameters
    ----------
    cont: int < totIa
        number of contaminant objects
    totIa: int (optional)
        Number of Ia in the sample. Default is 3000.
    before_salt2: bool (optional)
        If True use total sample number before SALT2 fit.
        Default is False.
    field: str (optional)
        Cadence: 'DDF' or 'WFD'. Default is DDF.
        
    Returns
    -------
    accuracy: floatTPR=TPR, FPR=FPR, FNR=FNR, TNR=TNR, TP=TP, FN=FN, TN=TN, FP=FP
    efficiency: float
    purity: float
    figure of merit (W=1): float
    figure of merit (W=3): float
    """
    
    if totIa != 3000:
        raise ValueError('Numbers are hard coded for 3000 SNIa.')
    
    total = {}
    total['DDF'] = {}                     # total number of objects in the sample
    total['DDF']['before_salt2'] = 4335
    total['DDF']['after_salt2'] = 3456
    total['WFD'] = {}
    total['WFD']['before_salt2'] = 5588
    total['WFD']['after_salt2'] = 3306
    
    if before_salt2:
        totall = total[field]['before_salt2']
    else:
        totall = total[field]['after_salt2']
    
    rate = nums_to_rate(tot=totall, P=totIa, T=totIa, FP=cont)._asdict()
    class_mets = det_mets(**rate)
    
#     acc = (totall - (2* totIa * cont))/totall
#     eff = (totIa - totIa * cont)/totIa
#     f1 = ((totIa - totIa * cont)/totIa) * (1 - cont)
#     f3 = ((1 - cont) * totIa)/(((1-cont) * totIa) + 3 * ((cont) * totIa))
    
#     return acc, eff, 1 - cont, f1, f3
    return class_mets.accuracy, class_mets.efficiency, class_mets.purity, class_mets.f1_score, class_mets.get_fom(3.)

In [ ]:
names = []

pop_Ia_all = []
pop_nIa_all = []
perc_Ia_all = []
perc_nIa_all = []

wfit_w_all = []
wfit_wsig_all = []
wfit_om_all = []
wfit_omsig_all = []

wfit_w_all_lowz = []
wfit_wsig_all_lowz = []
wfit_om_all_lowz = []
wfit_omsig_all_lowz = []

wdist_median = []

stan_w_all = []
stan_wsig_all = []
stan_om_all = []
stan_omsig_all = []

stan_w_all_lowz = []
stan_wsig_all_lowz = []
stan_om_all_lowz = []
stan_omsig_all_lowz = []

other_index = []
other_name = []

eff = []
pur = []
acc = []
f1 = []
f3 = []

for case in cases:
    
    print(case)

    names.append(case[:-4])
    
    pop = {}
    perc = {}

    samples_dir = case[:-4] + '/'
    
    data = pd.read_csv('/media/emille/git/COIN/RESSPECT_work/PLAsTiCC/metrics_paper/' +\
                       'resspect_metric/SALT2_fit/WFD' + str(k) + '/' + case)
    stats = np.unique(data['SIM_TYPE_INDEX'].values, return_counts=True)
   
    fname_cosmo_lowz = '/media/emille/git/COIN/RESSPECT_work/PLAsTiCC/metrics_paper/resspect_metric/' + \
                    'posteriors/WFD/' + case[:-4] + '/' + \
                    'test_mysamples' + str(k) + '/omprior_0.01_flat/results/test_salt2mu_lowz_withbias_' + case[:-4] + '.M0DIF.cospar'
    cosmofit_lowz = pd.read_csv(fname_cosmo_lowz, delim_whitespace=True,
                          comment='#', names=['w', 'wsig_marg',  'OM',  'OM_sig',  'chi2',  
                                              'Ndof',  'sigint', 'wran',  'OMran',  'label'])
    wfit_w_all_lowz.append(cosmofit_lowz['w'].values[0])
    wfit_wsig_all_lowz.append(cosmofit_lowz['wsig_marg'].values[0])
    wfit_om_all_lowz.append(cosmofit_lowz['OM'].values[0])
    wfit_omsig_all_lowz.append(cosmofit_lowz['OM_sig'].values[0])

    fname_stan = case[:-4] + '/test_mysamples' + str(k) + '/omprior_0.01_flat/results/stan_summary_' + case[:-4] + '_lowz_withbias.dat'
    op2 = open(fname_stan, 'r')
    lin2 = op2.readlines()
    op2.close()
    
    for j in range(len(lin2)):
        if lin2[j][:2] == 'om':
            om = lin2[j].split()[1]
            omsig = lin2[j].split()[3]
            stan_om_all_lowz.append(om)
            stan_omsig_all_lowz.append(omsig)
            
            #print('om = ', om)
            #print('omsig = ', omsig)
            
            
        elif lin2[j][0] == 'w':
            w = lin2[j].split()[1]
            wsig = lin2[j].split()[3]
            stan_w_all_lowz.append(w)
            stan_wsig_all_lowz.append(wsig)
            
    Ia_code = 11
        
    flag_Ia = np.array(stats[0]) == Ia_code
    
    pop[Ia_code] = stats[1][flag_Ia][0]
    perc[Ia_code] = round(100 * stats[1][flag_Ia][0]/data.shape[0], 2)
    
    if  len(stats[0]) == 2:
        other_code = [item for item in stats[0] if item != Ia_code][0]
        pop[other_code] = stats[1][~flag_Ia][0]
        perc[other_code] = round(100 * stats[1][~flag_Ia][0]/data.shape[0], 2)
            
        pop_nIa_all.append(pop[other_code])
        perc_nIa_all.append(perc[other_code])
        other_index.append(other_code)
        other_name.append(SNANA_names[other_code])
        
    elif len(stats[0]) > 2:
        other_code = [item for item in stats[0] if item != Ia_code]
        for item in range(flag_Ia.shape[0]):
            if not flag_Ia[item]:
                pop[stats[0][item]] = stats[1][item]
                perc[stats[0][item]] = round(100 * stats[1][item]/data.shape[0], 2)
                
        pop_nIa_all.append([pop[item] for item in other_code])
        perc_nIa_all.append([perc[item] for item in other_code])
        other_index.append(other_code)
        other_name.append([SNANA_names[i] for i in other_code])
        
    elif len(stats[0]) == 1:
        other_code = '--'
        pop_nIa_all.append(None)
        perc_nIa_all.append(None)
        other_index.append(None)
        other_name.append(None)

    pop_Ia_all.append(pop[Ia_code])
    perc_Ia_all.append(perc[Ia_code])
    
    if '6' in case:
        tot = 6000
    elif '5' in case:
        tot = 1500
    elif '1000' in case:
        tot = 1000
    else:
        tot = 3000
    
# this is where number of contaminants gets converted to fraction
    cont = tot - pop[Ia_code]
#     if '.' in case[:4]:
#         cont = 0.01 * (100 - float(case[:4]))
#     elif case[0] == 'p':
#         cont = 0
#     elif case[0] == 'r' or case[0] == 'f':
#         cont = (tot - pop[Ia_code])/tot
#     else:
#         cont = 0.01 * (100 - float(case[:2]))
    
    if k in [1, '', 2, 3,4,5]:
        for i in range(wdist_wfd.shape[0]):
            if case[:-4] in wdist_wfd['FileA'].values[i] or \
                case[:-4] in wdist_wfd['FileB'].values[i]:
                wdist_median.append(wdist_wfd['WassersteinDistanceMedian'].values[i])
                break
    else:
        wdist_median.append(-99)
    
    metrics = classification_metrics(cont)
    acc.append(metrics[0])
    eff.append(metrics[1])
    pur.append(metrics[2])
    f1.append(metrics[3]) 
    f3.append(metrics[4])
    
        
        
data_all = {}
data_all['case'] = names
data_all['other_name'] = other_name
data_all['other_code'] = other_index
data_all['nIa'] = pop_Ia_all
data_all['nothers'] = pop_nIa_all
data_all['perc_Ia'] = perc_Ia_all
data_all['perc_others'] = perc_nIa_all
data_all['accuracy'] = acc
data_all['efficiency'] = eff
data_all['purity'] = pur
data_all['fom1'] = f1
data_all['fom3'] = f3
data_all['wfit_w_lowz'] = wfit_w_all_lowz
data_all['wfit_wsig_lowz'] = wfit_wsig_all_lowz
data_all['wfit_om_lowz'] = wfit_om_all_lowz
data_all['wfit_omsig_lowz'] = wfit_omsig_all_lowz
data_all['stan_w_lowz'] = stan_w_all_lowz
data_all['stan_wsig_lowz'] = stan_wsig_all_lowz
data_all['stan_om_lowz'] = stan_om_all_lowz
data_all['stan_omsig_lowz'] = stan_omsig_all_lowz
data_all['WassersteinDistanceMedian'] = wdist_median

data_all = pd.DataFrame(data_all)

# add KLD
if k == '':
    fname1 = '/media/RESSPECT/data/PLAsTiCC/for_metrics/wfd/posteriors/klds.csv'
    data_kld_wfd = pd.read_csv(fname1)

    kld_wfd_column = []
    for i in range(data_all.shape[0]):
        name = data_all.iloc[i]['case']
    
        found = False
        for j in range(data_kld_wfd.shape[0]):
            if name in data_kld_wfd['path'].iloc[j]:
                kld = data_kld_wfd['KLD'].iloc[j]
                kld_wfd_column.append(kld)
                found = True
            
        if not found:
            kld_wfd_column.append(-99)
        
    data_all['KLD'] = kld_wfd_column
else:
    data_all['KLD'] = -99



In [ ]:
redo = True

if redo:
    data_all.to_csv('summary_cases_omprior_0.01_flat_redone' + str(k) +'.csv', index=False)
    data_all.to_csv('/media2/RESSPECT2/data/posteriors_wfd/omprior_0.01_flat/summary_cases_omprior_0.01_flat_redone' + \
                str(k) + '.csv', index=False)
else:
    data_all.to_csv('summary_cases_omprior_0.01_flat_emille' + str(k) +'.csv', index=False)
    data_all.to_csv('/media2/RESSPECT2/data/posteriors_wfd/omprior_0.01_flat/summary_cases_omprior_0.01_flat_emille' + \
                str(k) + '.csv', index=False)

In [ ]:
from shutil import copyfile

for name in cases:
    copyfile(name[:-4] + '/test_mysamples' + str(k) + '/omprior_0.01_flat/results/' + \
             'stan_input_salt2mu_lowz_withbias_' + name,
            '/media/RESSPECT/data/PLAsTiCC/for_metrics/ddf/distances/' + \
             'omprior_0.01_flat/emille_samples' + str(k) + '/stan_input_salt2mu_lowz_withbias_' + name)

In [ ]:
data_all = pd.read_csv('summary_cases.csv', index_col=False)

In [ ]:
flag_w = data_all['wfit_w'].values < 1000

plt.figure(figsize=(16,15))

plt.subplot(3,2,1)
plt.hist(data_all['wfit_w'][~flag_w], color='darkblue')
plt.hist(data_all['wfit_w'][flag_w], color='blue')
plt.xlabel('w_from_wfit', fontsize=14)
plt.ylabel('N', fontsize=14)


plt.subplot(3,2,2)
plt.hist(data_all['wfit_wsig'][~flag_w], color='darkblue')
plt.hist(data_all['wfit_wsig'][flag_w], color='blue')
plt.xlabel('wsig_from_wfit', fontsize=14)
plt.ylabel('N', fontsize=14)

plt.subplot(3,2,3)
plt.hist(data_all['stan_w'][~flag_w], color='b')
plt.hist(data_all['stan_w'][flag_w], color='darkblue')
plt.xlabel('w_from_stan', fontsize=14)
plt.ylabel('N', fontsize=14)

plt.subplot(3,2,4)
plt.hist(data_all['stan_wsig'][~flag_w], color='b')
plt.hist(data_all['stan_wsig'][flag_w], color='darkblue')
plt.xlabel('wsig_from_stan', fontsize=14)
plt.ylabel('N', fontsize=14)


plt.subplot(3,2,5)
plt.hist(data_all['stan_w_lowz'][~flag_w], color='green', alpha=0.5)
plt.hist(data_all['stan_w_lowz'][flag_w], color='brown')
plt.xlabel('w_from_stan_with_lowz', fontsize=14)
plt.ylabel('N', fontsize=14)

plt.subplot(3,2,6)
plt.hist(data_all['stan_wsig_lowz'][~flag_w], color='green', alpha=0.5)
plt.hist(data_all['stan_wsig_lowz'][flag_w], color='brown')
plt.xlabel('wsig_from_stan_with_lowz', fontsize=14)
plt.ylabel('N', fontsize=14)



plt.show()

In [ ]:
def highlight_col(x):
    r = 'background-color: pink'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 5] = r
    
    return df1    

In [ ]:
wdiff = data_all['wfit_w'].values - data_all['stan_w'].values

plt.figure(figsize=(16,5))

plt.subplot(1,2,1)
plt.hist(data_all['stan_w'][~flag_w], color='b')
plt.hist(data_all['stan_w'][flag_w], color='darkblue')
plt.xlabel('w_from_stan', fontsize=14)
plt.ylabel('N', fontsize=14)

plt.subplot(1,2,2)
plt.hist(wdiff[~flag_w], color='darkblue')
plt.hist(wdiff[flag_w], color='b')
plt.xlabel('wfit_w - stan_w', fontsize=14)
plt.ylabel('N', fontsize=4)

plt.show()

In [ ]:
from astropy.cosmology import FlatLambdaCDM
    
cosmo = FlatLambdaCDM(H0=72, Om0=0.3)
theor_dist = [cosmo.distmod(z).value for z in np.arange(0.001,1.5,0.005)]

for name in cases[:1]:
    
    fname_fitres = name[:-4] + '/results/test_salt2mu_' + name[:-4] + '.fitres'
    fitres = pd.read_csv(fname_fitres, comment='#', delim_whitespace=True)
    
    flag = fitres['SIM_TYPE_INDEX'].values == 11
    z = fitres['SIM_ZCMB'].values
    mu = fitres['MU'].values
    muerr = fitres['MUERR'].values
    
    
    fig = plt.figure(figsize=(10,8))
        
    ax = fig.add_subplot(111)
    ax.set_title(name, fontsize=26)

    if sum(flag) > 0:
        plt.errorbar(z[flag], mu[flag], yerr=muerr[flag], fmt='o', alpha=0.1, label='spec-Ia', color='blue')
    
    if sum(~flag) > 0:
        plt.errorbar(z[~flag], mu[~flag], yerr=muerr[~flag], fmt='^', alpha=0.1, label='photo-Ia', color='green')
       
    plt.plot(np.arange(0.001, 1.5,0.005), theor_dist, label='w = -1', color='red')

    w = str(cosmofit['w'].values[0])
    if len(w) >= 6:
        w1 = w[:6]
    else:
        w1 = w.ljust(6, '0')
            
    werr = str(cosmofit['wsig_marg'].values[0])
    if len(werr) >= 6:
        werr1 = werr[:5]
    else:
        werr1 = werr.ljust(5, '0')
            
    flag_case = data_all['case'].values == name[:-4]
    ax.text(0.2, 32, 'stan = ' + str(data_all[flag_case]['stan_w'].values[0]) + r' $\pm$ ' + str(data_all[flag_case]['stan_wsig'].values[0]), fontsize=20)
    ax.text(0.2, 30, r'wfit = ' + w1 + r' $\pm$ ' + werr1 , fontsize=20)
        
    ax.set_xlabel('redshift', fontsize=22)
    ax.set_ylabel('mu', fontsize=22)
    plt.legend(fontsize=22, loc='lower right')

    plt.savefig('plots/distances/dist_' + name[:-4] + '.png')
    plt.close('all')

In [ ]:
for name in cases:

    fname_fitres = name[:-4] + '/results/test_salt2mu_lowz_withbias_' + \
                   name[:-4] + '.fitres'
    fitres = pd.read_csv(fname_fitres, comment='#', delim_whitespace=True)
    
    flag = np.logical_or(fitres['SIM_TYPE_INDEX'].values == 11, 
                         fitres['SIM_TYPE_INDEX'].values == 1)
    z = fitres['SIM_ZCMB'].values
    mu = fitres['MU'].values
    muerr = fitres['MUERR'].values
    
    fig = plt.figure(figsize=(10,8))
        
    ax = fig.add_subplot(111)
    ax.set_title(name, fontsize=26)

    if sum(flag) > 0:
        #data2 = pd.concat([data[flag], lowz], ignore_index=True)
        plt.errorbar(z[flag], mu[flag], yerr=muerr[flag], fmt='o', 
                     alpha=0.1, label='spec-Ia', color='blue')
    
    if sum(~flag) > 0:
        plt.errorbar(z[~flag], mu[~flag], yerr=muerr[~flag], fmt='^', 
                     alpha=0.1, label='photo-Ia', color='green')
       
    plt.plot(np.arange(0.001, 1.5,0.005), theor_dist, label='w = -1', color='red')

    w = str(cosmofit_lowz['w'].values[0])
    if len(w) >= 6:
        w1 = w[:6]
    else:
        w1 = w.ljust(6, '0')
            
    werr = str(cosmofit_lowz['wsig_marg'].values[0])
    if len(werr) >= 6:
        werr1 = werr[:5]
    else:
        werr1 = werr.ljust(5, '0')
            
    flag_case = data_all['case'].values == name[:-4]
    ax.text(0.2, 32, 'stan = ' + str(data_all[flag_case]['stan_w_lowz'].values[0]) + \
            r' $\pm$ ' + \
            str(data_all[flag_case]['stan_wsig_lowz'].values[0]), fontsize=20)
    ax.text(0.2, 30, r'wfit = ' + w1 + r' $\pm$ ' + werr1 , fontsize=20)
        
    ax.set_xlabel('redshift', fontsize=22)
    ax.set_ylabel('mu', fontsize=22)
    plt.legend(fontsize=22, loc='lower right')

    plt.savefig('plots/distances/dist_' + name[:-4] + '_lowz_withbias.png')
    plt.close('all')

# Save samples for Alberto

In [ ]:
import pandas as pd
import os

k = 5

flist = os.listdir('.')
flist.remove('summary_cases_omprior_0.01_flat_emille.csv')
flist.remove('summary.ipynb')
flist.remove('plots')
flist.remove('.ipynb_checkpoints')
flist.remove('perfect3000_0')
flist.remove('perfect3000_I')
flist.remove('perfect3000_II')
flist.remove('perfect3000_III')
flist.remove('perfect3000_IV')
flist.remove('perfect3000_V')
flist.remove('perfect3000_VI')
flist.remove('perfect3000_VII')
flist.remove('perfect3000_VIII')
flist.remove('perfect3000_IX')
flist.remove('random1000')
flist.remove('random6000')
flist.remove('summary_cases.csv')
flist.remove('summary2.ipynb')
#flist.remove('99SNIa1SNIax')
for case in flist:

    if os.path.isdir(case + '/test_mysamples' + str(k) + '/omprior_0.01_flat/'):
        name = '/media/emille/git/COIN/RESSPECT_work/PLAsTiCC/metrics_paper/' + \
        'resspect_metric/posteriors/WFD/' + case + '/' + \
               'test_mysamples' + str(k) +'/omprior_0.01_flat/chains/chains_' + case + \
                '_lowz_withbias.pkl'
        data = pd.read_pickle(name)
        data2 = pd.DataFrame(data)
        print(case, ' --- ', data2.shape[0])

        data2.to_csv('/media/RESSPECT/data/PLAsTiCC/for_metrics/wfd/posteriors/' + \
                     'samples_emille' + str(k) + '/' + \
                    'chains_'  + case + '_lowz_withbias.csv.gz', index=False)
        
    else:
        print('**** ', case, '*****')